# Diffusion-Shock Inpainting in $SE(2)$
Diffusion-shock inpainting (DS) is a technique to fill in missing structures in images, developed in ["Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://link.springer.com/chapter/10.1007/978-3-031-31975-4_45) and the follow-up paper ["Regularised Diffusion-Shock Inpainting" (2023) by K. Schaefer and J. Weickert](https://arxiv.org/abs/2309.08761). In this notebook, we will look at DS applied to images lifted into $SE(2)$.

In $\mathbb{R}^2$, we can describe DS in a PDE-based formulation as
$$
\partial_t u = g(\lvert \nabla (G_{\nu} * u) \rvert^2) \underbrace{\Delta u}_{\textrm{Diffusion}} - \left(1 - g(\lvert \nabla (G_{\nu} * u) \rvert^2)\right) \underbrace{\mathrm{sgn}(\partial_{\vec{w} \vec{w}} (G_{\sigma} * u)) \lvert \nabla u \rvert}_{\textrm{Shock}},
$$
in which $g: [0, \infty) \to (0, 1]$ is a decreasing function with $g(0) = 1$, $G_{\alpha}$ is a Gaussian with standard deviation $\alpha$, and $\vec{w}$ is the dominant eigenvector of the structure tensor. It is clear then that $g$ switches between applying diffusion and shock: if the gradient of the image is small, we mostly apply diffusion, but if the gradient is large, we mostly apply shock. This makes sense, since a large gradient implies that there is a feature there, which we would like to sharpen up. 

The signum in the shock term switches between erosion and dilation. If the second derivative with respect to the dominant eigenvector of the structure tensor is positive, then we perform erosion (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$); otherwise we perform dilation (defined by the PDE $\partial_t u = -\lvert \nabla u \rvert$). In regularised DS, the signum is replaced with a soft signum, so that the selection of erosion vs dilation is less sensitive to noise.

The signum of the second derivative of the dominant eigenvector of the structure tensor is not unlike the convexity criterion we know from studying vesselness; perhaps we could replace it?

What is the correct way to extend DS to $SE(2)$? It would make sense to keep the gradients and Laplacian. For the selection of erosion vs dilation we could again look at the vesselness convexity criterion. For switching between diffusion and shock, we could maybe use some sort of line/edge detector.

## Setup

In [ ]:
import taichi as ti
ti.init(arch=ti.gpu, debug=False, device_memory_GB=3.5) #, kernel_profiler=True) # Use less than 4 so that we don't mix RAM and VRAM (?)
import numpy as np
from PIL import Image
from datetime import datetime
# from PIL import Image
import matplotlib.pyplot as plt
# %matplotlib widget
import dsfilter
from IPython.display import display, Markdown
from skimage.metrics import structural_similarity as ssim

In [2]:
date = datetime.today().strftime("%y-%m-%d")
log_folder = "..\\..\\Experiments For Paper\\R2 Enhancement"
savefigures = True
savedata = True

In [ ]:
date

In [4]:
def PSNR(denoised, ground_truth, max=255.):
    return 10 * np.log10(max**2 / np.mean((denoised - ground_truth)**2))

def L2(denoised, ground_truth):
    return np.sqrt(((denoised - ground_truth)**2).mean())

def L1(denoised, ground_truth):
    return np.abs(denoised - ground_truth).mean()

def SSIM(denoised, ground_truth, max=255.):
    return ssim(ground_truth, denoised, data_range=max)

In [5]:
# "spiral" "monalisa" "smoothspiral" "collagen"
test_case = "smoothspiral"

if test_case == "collagen":
    σ = 20.  # 20. 30.
    ρ = 0.
else:
    # σ = 0.2 * 255.  # Intensity
    # ρ = 1.          # Correlation
    # σ = 0.5 * 255.  # Intensity
    # ρ = 1.          # Correlation
    # σ = 0.5 * 255.  # Intensity
    # ρ = 2.          # Correlation
    σ = 1. * 255.   # Intensity
    ρ = 2.          # Correlation

In [ ]:
storage_name = f"{test_case}_{σ:.0f}"
if ρ > 0.:
    storage_name += f"_{ρ:.0f}"
storage_name

In [7]:
ζ = 1. # spatial anisotropy of metric.

In [8]:
dim_K = 16
ξ = 0.2

match test_case:
    case "spiral":
        ground_truth = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open("data/spiral.tif").convert("L")).astype(np.float64))
        noisy = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open(f"data/spiral_{σ:.0f}_{ρ:.0f}.tif")).astype(np.float64))
        # Diffusion-Shock Parameters.
        t_DS_SE2 = 2. * 2. / 25.
        
        t_DS_R2 = 2. * 10.
        
        # G_D_inv_gauge = 1.8 * np.array((1., ζ**2, 1.))
        G_D_inv_gauge = 1.8 * np.array((1., ζ**2, 1.))
        G_S_inv_gauge = np.array((1., ζ**2, 1.))
        # G_D_inv = 1.8 * np.array((ξ**-2, (ζ / ξ)**2, 1.))
        G_D_inv = 1.8 * np.array((ξ**-2, (ζ / ξ)**2, 1.))
        G_S_inv = np.array((ξ**-2, (ζ / ξ)**2, 1.))
        # Internal regularisation for switching between dilation and erosion.
        σ = 1.
        # External regularisation for switching between dilation and erosion.
        ρ = 1.
        # Internal and external regularisation of gradient for switching between diffusion and shock.
        ν = 1.

        λ_SE2 = 2.5
        ε_SE2 = 31.25
        λ_R2 = 2.
        ε_R2 = λ_R2 * 0.15
        # Total Variation Flow Parameters.
        t_TV = 1. * ξ # Looks good, but PSNR, L2, and L1 are better with shorter time 😢
        λ_TV = 50. / 255.
        # Plotting.
        im_width = 9
        im_height = 5
    case "monalisa":
        ground_truth = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open("data/monalisa.tif").convert("L")).astype(np.float64))
        noisy = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open(f"data/monalisa_{σ:.0f}_{ρ:.0f}.tif")).astype(np.float64))

        # Diffusion-Shock Parameters.
        t_DS_SE2 = 1. / 5.
        
        t_DS_R2 = 20.
        
        G_D_inv_gauge = 1.8 * np.array((1., ζ**2, 1.))
        G_S_inv_gauge = np.array((1., ζ**2, 1.))
        G_D_inv = 1.8 * np.array((ξ**-2, (ζ / ξ)**2, 1.))
        G_S_inv = np.array((ξ**-2, (ζ / ξ)**2, 1.))
        # Internal regularisation for switching between dilation and erosion.
        σ = 1.
        # External regularisation for switching between dilation and erosion.
        ρ = 1.
        # Internal and external regularisation of gradient for switching between diffusion and shock.
        ν = 1.

        λ_SE2 = 2.5
        ε_SE2 = 31.25
        λ_R2 = 2.
        ε_R2 = λ_R2 * 0.15
        # Total Variation Flow Parameters.
        t_TV = 1. * ξ # Looks good, but PSNR, L2, and L1 are better with shorter time 😢
        λ_TV = 50. / 255.
        # Plotting.
        im_width = 5
        im_height = 4
    case "smoothspiral":
        ground_truth = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open("data/smoothspiral.tif").convert("L")).astype(np.float64))
        noisy = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open(f"data/smoothspiral_{σ:.0f}_{ρ:.0f}.tif")).astype(np.float64))

        # Diffusion-Shock Parameters.
        t_DS_SE2 = 1. / 5.
        
        t_DS_R2 = 20.
        
        G_D_inv_gauge = 1.8 * np.array((1., ζ**2, 1.))
        G_S_inv_gauge = np.array((1., ζ**2, 1.))
        G_D_inv = 1.8 * np.array((ξ**-2, (ζ / ξ)**2, 1.))
        G_S_inv = np.array((ξ**-2, (ζ / ξ)**2, 1.))
        # Internal regularisation for switching between dilation and erosion.
        σ = 1.
        # External regularisation for switching between dilation and erosion.
        ρ = 1.
        # Internal and external regularisation of gradient for switching between diffusion and shock.
        ν = 1.

        λ_SE2 = 2.5
        ε_SE2 = 31.25
        λ_R2 = 2.
        ε_R2 = λ_R2 * 0.15
        # Total Variation Flow Parameters.
        t_TV = 1. * ξ # Looks good, but PSNR, L2, and L1 are better with shorter time 😢
        λ_TV = 50. / 255.
        # Plotting.
        im_width = 9
        im_height = 5
    case "collagen":
        ground_truth = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open("data/collagen.tif").convert("L")).astype(np.float64))
        noisy = dsfilter.SE2.utils.align_to_real_axis_scalar_field(np.array(Image.open(f"data/collagen_{σ:.0f}.tif")).astype(np.float64))

        # Diffusion-Shock Parameters.
        t_DS_SE2 = 1. / 5.
        
        t_DS_R2 = 20.
        
        G_D_inv_gauge = 1.8 * np.array((1., ζ**2, 1.))
        G_S_inv_gauge = np.array((1., ζ**2, 1.))
        G_D_inv = 1.8 * np.array((ξ**-2, (ζ / ξ)**2, 1.))
        G_S_inv = np.array((ξ**-2, (ζ / ξ)**2, 1.))
        # Internal regularisation for switching between dilation and erosion.
        σ = 1.
        # External regularisation for switching between dilation and erosion.
        ρ = 1.
        # Internal and external regularisation of gradient for switching between diffusion and shock.
        ν = 1.

        λ_SE2 = 2.5
        ε_SE2 = 31.25
        λ_R2 = 2.
        ε_R2 = λ_R2 * 0.15
        # Total Variation Flow Parameters.
        t_TV = 1. * ξ # Looks good, but PSNR, L2, and L1 are better with shorter time 😢
        λ_TV = 50. / 255.
        # Plotting.
        im_width = 6
        im_height = 5

# Arrays are aligned such that increasing the first index moves along the
# positive x-axis, and increasing the second index moves along the positive
# y-axis.

clip = (ground_truth.min(), ground_truth.max())

dim_I, dim_J = ground_truth.shape
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
PSNR_noisy = PSNR(noisy, ground_truth, max=255.)
L2_noisy = L2(noisy, ground_truth)
L1_noisy = L1(noisy, ground_truth)
SSIM_noisy = SSIM(noisy, ground_truth, max=255.)
PSNR_noisy, L2_noisy, L1_noisy, SSIM_noisy

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, 0., dim_I - 1., 0., dim_J - 1., fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, 0., dim_I - 1., 0., dim_J - 1., fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

### Orientation Score

In [11]:
cws = dsfilter.orientationscore.cakewavelet_stack(min(dim_I, dim_J), dim_K, Gaussian_σ=dim_I/32).real
U = dsfilter.orientationscore.wavelet_transform(noisy, cws).real
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ
mask = np.zeros_like(U) # Filtering, so there is no region outside of the mask

In [ ]:
K = 0
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(cws[K], x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1) - noisy, x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax);

In [15]:
zoomed = False
if zoomed:
    U = U[100:150, 100:150]
    dim_I, dim_J, dim_K = U.shape
    θs = θs[100:150, 100:150]
    ground_truth = ground_truth[100:150, 100:150]
    noisy = noisy[100:150, 100:150]

### Compute Gauge Frames

In [16]:
B1_LI, B2_LI, B3_LI, _ = dsfilter.SE2.gauge.frame.compute_gauge_frame_and_orientation_confidence(U, dxy, dθ, θs, ξ, ρ_s=1.5)
B1_static = dsfilter.SE2.utils.vectorfield_static_to_LI_np(B1_LI, θs)
B2_static = dsfilter.SE2.utils.vectorfield_static_to_LI_np(B2_LI, θs)
B3_static = dsfilter.SE2.utils.vectorfield_static_to_LI_np(B3_LI, θs)
gauge_frame_static = (B1_static, B2_static, B3_static)

## Perform Filtering

### TV Flow Filtering

#### Left Invariant

In [ ]:
U_TV_LI, PSNRs_TV_LI, L2s_TV_LI, L1s_TV_LI = dsfilter.TV_enhancing_LI(U, ground_truth, np.array((ξ**-2, ξ**-2, 1.)), dxy, dθ, θs, 1., 0.5, t_TV, λ=λ_TV)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(PSNRs_TV_LI)
ax[1].plot(L2s_TV_LI)
ax[2].plot(L1s_TV_LI);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_LI.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_LI.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Denoised")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(U_TV_LI.sum(-1), *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("Error");

In [ ]:
margin = 0.1
fig, ax, cbar = dsfilter.visualisations.plot_image_array((np.abs((ground_truth - np.clip(U_TV_LI.sum(-1), *clip))) > margin).astype(np.float64), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(np.abs((U - U_TV_LI)).max(-1), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
PSNR_TV_LI = PSNR(np.clip(U_TV_LI.sum(-1), *clip), ground_truth, max=255.)
L2_TV_LI = L2(np.clip(U_TV_LI.sum(-1), *clip), ground_truth)
L1_TV_LI = L1(np.clip(U_TV_LI.sum(-1), *clip), ground_truth)
SSIM_TV_LI = SSIM(np.clip(U_TV_LI.sum(-1), *clip), ground_truth, max=255.)
PSNR_TV_LI, L2_TV_LI, L1_TV_LI, SSIM_TV_LI

#### Gauge

In [ ]:
U_TV_gauge, PSNRs_TV_gauge, L2s_TV_gauge, L1s_TV_gauge = dsfilter.TV_enhancing_gauge(U, ground_truth, np.array((1., 1., 1.)), ξ, dxy, dθ, gauge_frame_static, 1., 0.5, t_TV, λ=λ_TV)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(PSNRs_TV_gauge)
ax[1].plot(L2s_TV_gauge)
ax[2].plot(L1s_TV_gauge);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Denoised")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(U_TV_gauge.sum(-1), *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("Error");

In [ ]:
margin = 0.1
fig, ax, cbar = dsfilter.visualisations.plot_image_array((np.abs((ground_truth - np.clip(U_TV_gauge.sum(-1), *clip))) > margin).astype(np.float64), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(np.abs((U - U_TV_gauge)).max(-1), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
PSNR_TV_gauge = PSNR(np.clip(U_TV_gauge.sum(-1), *clip), ground_truth, max=255.)
L2_TV_gauge = L2(np.clip(U_TV_gauge.sum(-1), *clip), ground_truth)
L1_TV_gauge = L1(np.clip(U_TV_gauge.sum(-1), *clip), ground_truth)
SSIM_TV_gauge = SSIM(np.clip(U_TV_gauge.sum(-1), *clip), ground_truth, max=255.)
PSNR_TV_gauge, L2_TV_gauge, L1_TV_gauge, SSIM_TV_gauge

### DS Filtering

#### Left Invariant

In [ ]:
U_DS, PSNRs_DS_LI, L2s_DS_LI, L1s_DS_LI, switch_DS, switch_morph = dsfilter.DS_enhancing_LI(U, ground_truth, θs, ξ, t_DS_SE2, G_D_inv, G_S_inv, σ, ρ, ν, λ_SE2, ε=ε_SE2, dxy=dxy)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(PSNRs_DS_LI)
ax[1].plot(L2s_DS_LI)
ax[2].plot(L1s_DS_LI);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Denoised")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(U_DS.sum(-1), *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("Error");

In [ ]:
K = 0
fig, ax = plt.subplots(1, 5, figsize=(5 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"$\\theta = {θs[0, 0, K + 2]:.2f}$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"$\\theta = {θs[0, 0, K + 4]:.2f}$")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS[..., K + 8], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[3])
ax[3].set_title(f"$\\theta = {θs[0, 0, K + 8]:.2f}$")
fig.colorbar(cbar, ax=ax[3])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, cmap="gray", fig=fig, ax=ax[4])
ax[4].set_title("$\int_\\theta U(\\theta) d\\theta$")
fig.colorbar(cbar, ax=ax[4]);

In [ ]:
K = 0
fig, ax = plt.subplots(3, 2, figsize=(2 * im_width, 3 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$\\theta = {θs[0, 0, K + 2]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$\\theta = {θs[0, 0, K + 4]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 8], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$\\theta = {θs[0, 0, K + 8]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title("$\int_\\theta d\\theta$")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title("$\min_\\theta$")
fig.colorbar(cbar, ax=ax[2, 1]);

In [ ]:
K = 0
fig, ax = plt.subplots(3, 2, figsize=(2 * im_width, 3 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$\\theta = {θs[0, 0, K + 2]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$\\theta = {θs[0, 0, K + 4]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K + 8], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$\\theta = {θs[0, 0, K + 8]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title("$\int_\\theta d\\theta$")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title("$\min_\\theta$")
fig.colorbar(cbar, ax=ax[2, 1]);

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(4 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
ax[0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
ax[1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2])
ax[2].set_title("Denoised")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(U_DS.sum(-1), *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[3])
fig.colorbar(cbar, ax=ax[3])
ax[3].set_title("Error");

In [ ]:
margin = 0.1
fig, ax, cbar = dsfilter.visualisations.plot_image_array((np.abs((ground_truth - np.clip(U_DS.sum(-1), *clip))) > margin).astype(np.float64), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
PSNR_DS = PSNR(np.clip(U_DS.sum(-1), *clip), ground_truth, max=255.)
L2_DS = L2(np.clip(U_DS.sum(-1), *clip), ground_truth)
L1_DS = L1(np.clip(U_DS.sum(-1), *clip), ground_truth)
SSIM_DS = SSIM(np.clip(U_DS.sum(-1), *clip), ground_truth, max=255.)
PSNR_DS, L2_DS, L1_DS, SSIM_DS

#### Gauge

In [ ]:
U_DS_gauge, PSNRs_DS_gauge, L2s_DS_gauge, L1s_DS_gauge, switch_DS, switch_morph = dsfilter.DS_enhancing_gauge(U, ground_truth, θs, ξ, gauge_frame_static, t_DS_SE2, G_D_inv_gauge, G_S_inv_gauge, σ, ρ, ν, λ_SE2, ε=ε_SE2, dxy=dxy)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(PSNRs_DS_gauge)
ax[1].plot(L2s_DS_gauge)
ax[2].plot(L1s_DS_gauge);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Denoised")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(U_DS_gauge.sum(-1), *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("Error");

In [ ]:
K = 0
fig, ax = plt.subplots(1, 5, figsize=(5 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"$\\theta = {θs[0, 0, K + 2]:.2f}$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"$\\theta = {θs[0, 0, K + 4]:.2f}$")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge[..., K + 8], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[3])
ax[3].set_title(f"$\\theta = {θs[0, 0, K + 8]:.2f}$")
fig.colorbar(cbar, ax=ax[3])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, cmap="gray", fig=fig, ax=ax[4])
ax[4].set_title("$\int_\\theta U(\\theta) d\\theta$")
fig.colorbar(cbar, ax=ax[4]);

In [ ]:
K = 0
fig, ax = plt.subplots(3, 2, figsize=(2 * im_width, 3 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$\\theta = {θs[0, 0, K + 2]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$\\theta = {θs[0, 0, K + 4]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 8], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$\\theta = {θs[0, 0, K + 8]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title("$\int_\\theta d\\theta$")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title("$\min_\\theta$")
fig.colorbar(cbar, ax=ax[2, 1]);

In [ ]:
K = 0
fig, ax = plt.subplots(3, 2, figsize=(2 * im_width, 3 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$\\theta = {θs[0, 0, K]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"$\\theta = {θs[0, 0, K + 2]:.2f}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$\\theta = {θs[0, 0, K + 4]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K + 8], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"$\\theta = {θs[0, 0, K + 8]:.2f}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title("$\int_\\theta d\\theta$")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS.min(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title("$\min_\\theta$")
fig.colorbar(cbar, ax=ax[2, 1]);

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(4 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
ax[0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
ax[1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2])
ax[2].set_title("Denoised")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(U_DS_gauge.sum(-1), *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[3])
fig.colorbar(cbar, ax=ax[3])
ax[3].set_title("Error");

In [ ]:
margin = 0.1
fig, ax, cbar = dsfilter.visualisations.plot_image_array((np.abs((ground_truth - np.clip(U_DS_gauge.sum(-1), *clip))) > margin).astype(np.float64), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
PSNR_DS_gauge = PSNR(np.clip(U_DS_gauge.sum(-1), *clip), ground_truth, max=255.)
L2_DS_gauge = L2(np.clip(U_DS_gauge.sum(-1), *clip), ground_truth)
L1_DS_gauge = L1(np.clip(U_DS_gauge.sum(-1), *clip), ground_truth)
SSIM_DS_gauge = SSIM(np.clip(U_DS_gauge.sum(-1), *clip), ground_truth, max=255.)
PSNR_DS_gauge, L2_DS_gauge, L1_DS_gauge, SSIM_DS_gauge

In [ ]:
PSNR_DS, L2_DS, L1_DS, SSIM_DS

#### $\mathbb{R}^2$

In [ ]:
PSNR(np.clip(U.sum(-1), 0., 255.), ground_truth)

In [ ]:
PSNR(noisy, ground_truth)

In [ ]:
u_DS_R2, PSNRs_DS_R2, L2s_DS_R2, L1s_DS_R2, switch_DS, switch_morph = dsfilter.DS_enhancing_R2(noisy, ground_truth, t_DS_R2, σ, ρ, ν, λ_SE2, ε=ε_SE2, dxy=dxy)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(PSNRs_DS_R2)
ax[1].plot(L2s_DS_R2)
ax[2].plot(L1s_DS_R2);

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_DS_R2, x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1])
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_DS_R2, x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Denoised")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(u_DS_R2, *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("Error");

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * im_width, 1 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("Morphological Switch")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("DS Switch")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(4 * im_width, im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0])
ax[0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[1])
ax[1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_DS_R2, x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2])
ax[2].set_title("Denoised")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth - np.clip(u_DS_R2, *clip), x_min, x_max, y_min, y_max, fig=fig, ax=ax[3])
fig.colorbar(cbar, ax=ax[3])
ax[3].set_title("Error");

In [ ]:
margin = 0.1
fig, ax, cbar = dsfilter.visualisations.plot_image_array((np.abs((ground_truth - np.clip(u_DS_R2, *clip))) > margin).astype(np.float64), x_min, x_max, y_min, y_max, figsize=(im_width, im_height))
fig.colorbar(cbar, ax=ax)
ax.set_title(f"Error > {margin}");

In [ ]:
PSNR_DS_R2 = PSNR(u_DS_R2, ground_truth, max=255.)
L2_DS_R2 = L2(u_DS_R2, ground_truth)
L1_DS_R2 = L1(u_DS_R2, ground_truth)
SSIM_DS_R2 = SSIM(u_DS_R2, ground_truth, max=255.)
PSNR_DS_R2, L2_DS_R2, L1_DS_R2, SSIM_DS_R2

### Comparison

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_LI.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("TV Flow")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("DS Filtering");

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * im_width, 2 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_DS_R2, x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("$\mathbb{R}^2$ DS Filtering")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("$\mathrm{SE}(2)$ DS Filtering");

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(2 * im_width, 3 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_LI.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Left Invariant TV Flow")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("Gauge TV Flow")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_DS_R2, x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2, 0])
fig.colorbar(cbar, ax=ax[2, 0])
ax[2, 0].set_title("$\mathbb{R}^2$ DS Filtering")
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2, 1])
fig.colorbar(cbar, ax=ax[2, 1])
ax[2, 1].set_title("$\mathrm{SE}(2)$ DS Filtering");

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(2 * im_width, 3 * im_height))
_, _, cbar = dsfilter.visualisations.plot_image_array(ground_truth, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Ground Truth")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(noisy, x_min, x_max, y_min, y_max, fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Noisy")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_TV_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Gauge TV Flow")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u_DS_R2, x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[1, 1])
fig.colorbar(cbar, ax=ax[1, 1])
ax[1, 1].set_title("$\mathbb{R}^2$ DS Filtering")
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2, 0])
fig.colorbar(cbar, ax=ax[2, 0])
ax[2, 0].set_title("Left Invariant DS Filtering")
_, _, cbar = dsfilter.visualisations.plot_image_array(U_DS_gauge.sum(-1), x_min, x_max, y_min, y_max, clip=clip, fig=fig, ax=ax[2, 1])
fig.colorbar(cbar, ax=ax[2, 1])
ax[2, 1].set_title("Gauge DS Filtering");

In [66]:
if savefigures:
    fig.savefig(f"{log_folder}\\{test_case}\\{storage_name}_{date}.svg", bbox_inches="tight", dpi=200)

In [67]:
ts_TV_LI = np.linspace(0, 1, len(PSNRs_TV_LI))
ts_TV_gauge = np.linspace(0, 1, len(PSNRs_TV_gauge))
ts_DS_LI = np.linspace(0, 1, len(PSNRs_DS_LI))
ts_DS_gauge = np.linspace(0, 1, len(PSNRs_DS_gauge))
ts_DS_R2 = np.linspace(0, 1, len(PSNRs_DS_R2))

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].plot(ts_TV_LI, PSNRs_TV_LI, label="TV LI", color="red")
ax[0].plot(ts_TV_gauge, PSNRs_TV_gauge, label="TV Gauge", color="red", linestyle="dashed")
ax[0].plot(ts_DS_LI, PSNRs_DS_LI, label="DS LI", color="green")
ax[0].plot(ts_DS_gauge, PSNRs_DS_gauge, label="DS Gauge", color="green", linestyle="dashed")
ax[0].plot(ts_DS_R2, PSNRs_DS_R2, label="DS $\mathbb{R}^2$", color="blue")
ax[0].set_title("PSNR")
ax[0].set_xlim(0., 1.)
ax[0].set_xlabel("$\\tilde{t}$")
ax[0].set_ylabel("PSNR (dB)")
ax[0].legend()
ax[1].plot(ts_TV_LI, L2s_TV_LI, label="TV LI", color="red")
ax[1].plot(ts_TV_gauge, L2s_TV_gauge, label="TV Gauge", color="red", linestyle="dashed")
ax[1].plot(ts_DS_LI, L2s_DS_LI, label="DS LI", color="green")
ax[1].plot(ts_DS_gauge, L2s_DS_gauge, label="DS Gauge", color="green", linestyle="dashed")
ax[1].plot(ts_DS_R2, L2s_DS_R2, label="DS $\mathbb{R}^2$", color="blue")
ax[1].set_title("$\mathbb{L}_2$ Error")
ax[1].set_xlim(0., 1.)
ax[1].set_xlabel("$\\tilde{t}$")
ax[1].set_ylabel("$\Vert u - \\text{ground truth} \Vert_2$")
ax[1].legend()
ax[2].plot(ts_TV_LI, L1s_TV_LI, label="TV LI", color="red")
ax[2].plot(ts_TV_gauge, L1s_TV_gauge, label="TV Gauge", color="red", linestyle="dashed")
ax[2].plot(ts_DS_LI, L1s_DS_LI, label="DS LI", color="green")
ax[2].plot(ts_DS_gauge, L1s_DS_gauge, label="DS Gauge", color="green", linestyle="dashed")
ax[2].plot(ts_DS_R2, L1s_DS_R2, label="DS $\mathbb{R}^2$", color="blue")
ax[2].set_title("$\mathbb{L}_1$ Error")
ax[2].set_xlim(0., 1.)
ax[2].set_xlabel("$\\tilde{t}$")
ax[2].set_ylabel("$\Vert u - \\text{ground truth} \Vert_1$")
ax[2].legend();

In [69]:
if savefigures:
    fig.savefig(f"{log_folder}\\{test_case}\\{storage_name}_quality_measures_{date}.svg", bbox_inches="tight", dpi=200)

In [70]:
if savedata:
    data_PSNRs_TV_LI = np.array((ts_TV_LI, PSNRs_TV_LI)).transpose()
    data_PSNRs_TV_gauge = np.array((ts_TV_gauge, PSNRs_TV_gauge)).transpose()
    data_PSNRs_DS_LI = np.array((ts_DS_LI, PSNRs_DS_LI)).transpose()
    data_PSNRs_DS_gauge = np.array((ts_DS_gauge, PSNRs_DS_gauge)).transpose()
    data_PSNRs_DS_R2 = np.array((ts_DS_R2, PSNRs_DS_R2)).transpose()
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_PSNR_TV_LI.csv", data_PSNRs_TV_LI, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_PSNR_TV_gauge.csv", data_PSNRs_TV_gauge, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_PSNR_DS_LI.csv", data_PSNRs_DS_LI, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_PSNR_DS_gauge.csv", data_PSNRs_DS_gauge, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_PSNR_DS_R2.csv", data_PSNRs_DS_R2, delimiter=",")
    
    data_L2s_TV_LI = np.array((ts_TV_LI, L2s_TV_LI)).transpose()
    data_L2s_TV_gauge = np.array((ts_TV_gauge, L2s_TV_gauge)).transpose()
    data_L2s_DS_LI = np.array((ts_DS_LI, L2s_DS_LI)).transpose()
    data_L2s_DS_gauge = np.array((ts_DS_gauge, L2s_DS_gauge)).transpose()
    data_L2s_DS_R2 = np.array((ts_DS_R2, L2s_DS_R2)).transpose()
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L2_TV_LI.csv", data_L2s_TV_LI, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L2_TV_gauge.csv", data_L2s_TV_gauge, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L2_DS_LI.csv", data_L2s_DS_LI, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L2_DS_gauge.csv", data_L2s_DS_gauge, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L2_DS_R2.csv", data_L2s_DS_R2, delimiter=",")
    
    data_L1s_TV_LI = np.array((ts_TV_LI, L1s_TV_LI)).transpose()
    data_L1s_TV_gauge = np.array((ts_TV_gauge, L1s_TV_gauge)).transpose()
    data_L1s_DS_LI = np.array((ts_DS_LI, L1s_DS_LI)).transpose()
    data_L1s_DS_gauge = np.array((ts_DS_gauge, L1s_DS_gauge)).transpose()
    data_L1s_DS_R2 = np.array((ts_DS_R2, L1s_DS_R2)).transpose()
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L1_TV_LI.csv", data_L1s_TV_LI, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L1_TV_gauge.csv", data_L1s_TV_gauge, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L1_DS_LI.csv", data_L1s_DS_LI, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L1_DS_gauge.csv", data_L1s_DS_gauge, delimiter=",")
    np.savetxt(f"{log_folder}\\data\\{test_case}\\{storage_name}_L1_DS_R2.csv", data_L1s_DS_R2, delimiter=",")


In [ ]:
# Generate Markdown
markdown_content = f"""
| Denoiser | PSNR | $\mathbb{{L}}_1$ | $\mathbb{{L}}_2$ | SSIM |
|:-:|-:|-:|-:|-:|
| - | {PSNR_noisy:.2f} | {L1_noisy:.2f} | {L2_noisy:.2f} | {SSIM_noisy:.3f} |
| LI TV | {PSNR_TV_LI:.2f} | {L1_TV_LI:.2f} | {L2_TV_LI:.2f} | {SSIM_TV_LI:.3f} |
| Gauge TV | {PSNR_TV_gauge:.2f} | {L1_TV_gauge:.2f} | {L2_TV_gauge:.2f} | {SSIM_TV_gauge:.3f} |
| $\mathbb{{R}}^2$ DS | {PSNR_DS_R2:.2f} | {L1_DS_R2:.2f} | {L2_DS_R2:.2f} | {SSIM_DS_R2:.3f} |
| LI DS | {PSNR_DS:.2f} | {L1_DS:.2f} | {L2_DS:.2f} | {SSIM_DS:.3f} |
| Gauge DS | {PSNR_DS_gauge:.2f} | {L1_DS_gauge:.2f} | {L2_DS_gauge:.2f} | {SSIM_DS_gauge:.3f} |
"""

# Display Markdown
display(Markdown(markdown_content))

Mona Lisa

| Denoiser | PSNR | $\mathbb{L}_1$ | $\mathbb{L}_2$ | SSIM |
|:-:|-:|-:|-:|-:|
| - | 24.53 | 11.96 | 15.14 | 0.560 |
| LI TV | 26.95 | 8.50 | 11.45 | 0.627 |
| Gauge TV | 26.94 | 8.52 | 11.47 | 0.625 |
| $\mathbb{R}^2$ DS | 26.04 | 9.32 | 12.72 | 0.553 |
| SE(2) DS | 26.16 | 9.22 | 12.54 | 0.560 |

Spiral

| Denoiser | PSNR | $\mathbb{L}_1$ | $\mathbb{L}_2$ | SSIM |
|:-:|-:|-:|-:|-:|
| - | 26.30 | 8.09 | 12.34 | 0.719 |
| LI TV | 23.10 | 13.08 | 17.85 | 0.915 |
| Gauge TV | 23.18 | 12.89 | 17.69 | 0.912 |
| $\mathbb{R}^2$ DS | 23.49 | 7.81 | 17.07 | 0.948 |
| SE(2) DS | 25.75 | 7.70 | 13.16 | 0.944 |